In [1]:
import torch
# from diffusers import DiffusionPipeline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [2]:
# pipe = DiffusionPipeline.from_pretrained("Hyeon2/riffusion-musiccaps", torch_dtype=torch.float16).to(device)

# prompt = "a jazz solo, enegetic with syncopation"
# image = pipe(prompt).images[0]

In [3]:
from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_params import SpectrogramParams
from pydub import AudioSegment

params = SpectrogramParams(
    stereo = True,
    max_frequency=20000,
)
converter = SpectrogramImageConverter(params)

audio_path = r"D:\Documents\OctAI\Text2Music\downloaded_files\electro\2 Beers and a Good Send.mp3"
audio_segment = AudioSegment.from_mp3(audio_path)

#5110 ms generate image of 512x512
img = converter.spectrogram_image_from_audio(audio_segment[0:5110])

In [4]:
def chunk_audio_by_ms(audio_segment, ms):
    chunks = []
    for i in range(0, len(audio_segment), ms):
        chunks.append(audio_segment[i:i+ms])
    return chunks


In [5]:
def audio_chunks_to_images(chunks, converter):
    images = []
    for chunk in chunks:
        images.append(converter.spectrogram_image_from_audio(chunk))
    return images

In [6]:
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm

In [40]:
def get_row_by_titles(df, titles):
    return df.loc[df["title"].isin(titles)].drop_duplicates()

def get_all_files_in_folder(path):
    return [f"{path}/{file}" for file in os.listdir(path)]

def get_title_from_path(path):
    return "".join(path.split("/")[-1].split(".", maxsplit=1))

def string_list_to_list(string):
    # transform a string of list to an actual list in python
    replaced = string[1:-1].replace("'", "")
    if "," in replaced:
        return replaced.split(", ")
    else:
        return [replaced]

In [41]:
audio_folder = r"D:\Documents\OctAI\Text2Music\downloaded_files\electro"
epi_csv = r"D:\Documents\OctAI\Text2Music\epidemic_music.csv"

df = pd.read_csv(epi_csv)

In [42]:
titles = [get_title_from_path(path) for path in get_all_files_in_folder(audio_folder)]

In [43]:
filtered_df = get_row_by_titles(df, titles)

filtered_df.genres = filtered_df.genres.apply(string_list_to_list)
filtered_df.metadataTags = filtered_df.metadataTags.apply(string_list_to_list)
filtered_df.moods = filtered_df.moods.apply(string_list_to_list)

In [44]:
def generate_pseudo_label(df):
    return ",".join(df["genres"]) + "," + ",".join(df["metadataTags"]) + "," + ",".join(df["moods"])

In [45]:
filtered_df["pseudo_label"] = filtered_df.apply(generate_pseudo_label, axis=1)

In [46]:
filtered_df

,title,id,added,length,bpm,isSfx,hasVocals,energyLevel,genres,url,metadataTags,moods,pseudo_label
25894,Next,22373.0,2012-06-25T09:43:07,134.0,85.0,0.0,0.0,medium,"[beautiful, drama]",https://dkihjuum4jcjr.cloudfront.net/ES_ITUNES...,"[road, waiting, family, rain, morning, family,...","[happy, romantic]","beautiful,drama,road,waiting,family,rain,morni..."


In [34]:
data = []
for tlt in tqdm(titles):
    try:
        audio_path = os.path.join(audio_folder, tlt + ".mp3")
        audio_segment = AudioSegment.from_mp3(audio_path)
        chunks = chunk_audio_by_ms(audio_segment, 5110)
        spectrograms = audio_chunks_to_images(chunks[:-1], converter)  # last chunk is too short
        # print(spectrograms)
        row = filtered_df.loc[filtered_df["title"] == tlt]
        pseudo_label = row["pseudo_label"].values[0]
        song_title = row["title"].values[0]

        data += [(song_title, pseudo_label, spectro) for spectro in spectrograms]
    except Exception as e:
        print()

  0%|          | 0/1168 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Documents\\OctAI\\Text2Music\\downloaded_files\\electro\\Dot.mp3'

In [31]:
data[0][1]

'electro,traditional-jazz,bouncy,electronic,sophisticated,scat,electro swing,jive,bar,cocktails,swinging,sexy,dancing,hip,fashion,contemporary,luxurious ,manouche,lounge,busy-frantic'

In [39]:
"".join("dot..".split(".", maxsplit=1))

'dot.'